##Preliminary Notes

The aim of the InCrediblAE shared task is to build your own custom attack method that will generate adversarial examples to fool a victim classifier. This notebook is intended as an easy way for you to get started.

<br>

### Using GPU
It is recommended that you run this notebook with a GPU. To do this, click on "additional connection options" (next to Connect / RAM usage), select "change runtime type", and select a GPU.

<br>

### (optional) Mounting Google Drive - don't bother with this if running this notebook for first time
If you will be re-running this notebook many times, it might be convenient to mount your personal google drive. This will allow you to
1. load data/victim files quickly rather than re-downloading them with each session
2. save output files to a permanent location

Instructions for mounting are in the 'Making your own attack section'.


# Setup (installing dependencies)

In [1]:
!git clone https://github.com/piotrmp/BODEGA

Cloning into 'BODEGA'...


In [2]:
%pip install OpenAttack
%pip install editdistance
%pip install bert-score
%pip install git+https://github.com/lucadiliello/bleurt-pytorch.git


  Using cached OpenAttack-2.1.1-py3-none-any.whl.metadata (20 kB)
Using cached OpenAttack-2.1.1-py3-none-any.whl (145 kB)
   ---------------------------------------- 0.0/542.0 kB ? eta -:--:--
   -- ------------------------------------- 30.7/542.0 kB 1.4 MB/s eta 0:00:01
   ------ --------------------------------- 81.9/542.0 kB 1.2 MB/s eta 0:00:01
   ----------- ---------------------------- 153.6/542.0 kB 1.3 MB/s eta 0:00:01
   --------------------- ------------------ 297.0/542.0 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 542.0/542.0 kB 2.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/116.3 kB ? eta -:--:--
   ---------------------------------------- 116.3/116.3 kB ? eta 0:00:00
   ---------------------------------------- 0.0/25.9 MB ? eta -:--:--
    --------------------------------------- 0.5/25.9 MB 31.4 MB/s eta 0:00:01
   - -------------------------------------- 1.2/25.9 MB 15.6 MB/s eta 0:00:02
   -- ---------------------------------

  Running command git clone --filter=blob:none --quiet https://github.com/lucadiliello/bleurt-pytorch.git 'C:\Users\selin\AppData\Local\Temp\pip-req-build-ichsvm4i'


In [3]:
!git clone https://gitlab.clarin-pl.eu/syntactic-tools/lambo.git
%pip install ./lambo


Cloning into 'lambo'...


Processing c:\users\selin\documents\checkthat\\lambo
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for lambo: filename=lambo-2.3-py3-none-any.whl size=93748 sha256=b3a6074780efcc30ad00b9e1b11f141887e26ad1b439d6e9834e7371d728ab13
  Stored in directory: C:\Users\selin\AppData\Local\Temp\pip-ephem-wheel-cache-0rlu5rbn\wheels\77\c2\fd\2f3d7cc132797ecc6494f67dbd66b5a3e9c994a2fdbd2ae181
Successfully built lambo
Note: you may need to restart the kernel to use updated packages.


# Downloading victim models and data

Data and models are downloaded by cloning the [clef2024-checkthat repo](https://gitlab.com/checkthat_lab/clef2024-checkthat-lab.git)
* alternative [google drive folder link](https://drive.google.com/drive/folders/1ZsDHSejiv4USae0viTsfeLpvqXdeq0FL?usp=sharing)

Data and models are downloaded then moved to /content/BODEGA/incrediblAE_public_release

In [4]:
# temporary folder for downloading victim models and data
! mkdir clef2024-checkthat-lab

import os, sys
os.chdir("clef2024-checkthat-lab")

! git init
! git remote add -f origin https://gitlab.com/checkthat_lab/clef2024-checkthat-lab.git
! git sparse-checkout init
! git sparse-checkout set "task6/incrediblAE_public_release"
! git pull origin main


Initialized empty Git repository in C:/Users/selin/Documents/CheckThat/clef2024-checkthat-lab/.git/
Updating origin


From https://gitlab.com/checkthat_lab/clef2024-checkthat-lab
 * [new branch]      FHaouari-main-patch-26640 -> origin/FHaouari-main-patch-26640
 * [new branch]      FHaouari-main-patch-34250 -> origin/FHaouari-main-patch-34250
 * [new branch]      FHaouari-main-patch-81200 -> origin/FHaouari-main-patch-81200
 * [new branch]      main       -> origin/main
From https://gitlab.com/checkthat_lab/clef2024-checkthat-lab
 * branch            main       -> FETCH_HEAD
Updating files:  22% (8/35)
Updating files:  25% (9/35)
Updating files:  28% (10/35)
Updating files:  31% (11/35)
Updating files:  34% (12/35)
Updating files:  37% (13/35)
Updating files:  40% (14/35)
Updating files:  42% (15/35)
Updating files:  45% (16/35)
Updating files:  48% (17/35)
Updating files:  51% (18/35)
Updating files:  54% (19/35)
Updating files:  57% (20/35)
Updating files:  60% (21/35)
Updating files:  62% (22/35)
Updating files:  65% (23/35)
Updating files:  68% (24/35)
Updating files:  71% (25/35)
Updating files: 

In [5]:
# move downloaded files to /content/BODEGA
! mv clef2024-checkthat-lab/task6/incrediblAE_public_release BODEGA/incrediblAE_public_release

'mv' is not recognized as an internal or external command,
operable program or batch file.


Misc set up

In [3]:
#folder for storing results of attack method
! mkdir BODEGA/outputs

#code below assumes we are working from the BODEGA repo
os.chdir("./BODEGA")

The syntax of the command is incorrect.


# Making your own attack

## Imports

In [4]:
import gc
import os
import pathlib
import sys
import time
import random
import numpy as np

import OpenAttack
import torch
import datasets
from datasets import Dataset

from OpenAttack.tags import Tag
from OpenAttack.text_process.tokenizer import PunctTokenizer

from metrics.BODEGAScore import BODEGAScore
from utils.data_mappings import dataset_mapping, dataset_mapping_pairs, SEPARATOR_CHAR
from utils.no_ssl_verify import no_ssl_verify
from victims.bert import VictimBERT, readfromfile_generator
from victims.bilstm import VictimBiLSTM
from victims.caching import VictimCache
from victims.unk_fix_wrapper import UNK_TEXT

#imports for BodegaAttackEval wrapper
from typing import Any, Dict, Generator, Iterable, List, Optional, Union
from tqdm import tqdm
from OpenAttack.utils import visualizer, result_visualizer, get_language, language_by_name
from OpenAttack.tags import *

In [5]:
using_mounted_drive = False
print('Cuda device available', torch.cuda.is_available())

Cuda device available True


## (do not change) Wrapper for producing submission file

In [6]:
class BodegaAttackEval(OpenAttack.AttackEval):
  '''
  wrapper for OpenAttack.AttackEval to produce a submission.tsv file for shared task evaluation

  To perform evaluation, we use a new method: eval_and_save_tsv() rather than the usual AttackEval.eval()
  submission.tsv file consists of 4 columns for each sample in attack set: succeeded, num_queries, original_text and modified text (newlines are escaped)

  '''
  def eval_and_save_tsv(self, dataset: Iterable[Dict[str, Any]], total_len : Optional[int] = None, visualize : bool = False, progress_bar : bool = False, num_workers : int = 0, chunk_size : Optional[int] = None, tsv_file_path: Optional[os.PathLike] = None):
      """
      Evaluation function of `AttackEval`.

      Args:
          dataset: An iterable dataset.
          total_len: Total length of dataset (will be used if dataset doesn't has a `__len__` attribute).
          visualize: Display a pretty result for each data in the dataset.
          progress_bar: Display a progress bar if `True`.
          num_workers: The number of processes running the attack algorithm. Default: 0 (running on the main process).
          chunk_size: Processing pool trunks size.

          tsv_file_path: path to save submission tsv

      Returns:
          A dict of attack evaluation summaries.

      """


      if hasattr(dataset, "__len__"):
          total_len = len(dataset)

      def tqdm_writer(x):
          return tqdm.write(x, end="")

      if progress_bar:
          result_iterator = tqdm(self.ieval(dataset, num_workers, chunk_size), total=total_len)
      else:
          result_iterator = self.ieval(dataset, num_workers, chunk_size)

      total_result = {}
      total_result_cnt = {}
      total_inst = 0
      success_inst = 0

      #list for tsv
      x_orig_list = []
      x_adv_list = []
      num_queries_list = []
      succeed_list = []

      # Begin for
      for i, res in enumerate(result_iterator):
          total_inst += 1
          success_inst += int(res["success"])

          if TAG_Classification in self.victim.TAGS:
              x_orig = res["data"]["x"]
              if res["success"]:
                  x_adv = res["result"]
                  if Tag("get_prob", "victim") in self.victim.TAGS:
                      self.victim.set_context(res["data"], None)
                      try:
                          probs = self.victim.get_prob([x_orig, x_adv])
                      finally:
                          self.victim.clear_context()
                      y_orig = probs[0]
                      y_adv = probs[1]
                  elif Tag("get_pred", "victim") in self.victim.TAGS:
                      self.victim.set_context(res["data"], None)
                      try:
                          preds = self.victim.get_pred([x_orig, x_adv])
                      finally:
                          self.victim.clear_context()
                      y_orig = int(preds[0])
                      y_adv = int(preds[1])
                  else:
                      raise RuntimeError("Invalid victim model")
              else:
                  y_adv = None
                  x_adv = None
                  if Tag("get_prob", "victim") in self.victim.TAGS:
                      self.victim.set_context(res["data"], None)
                      try:
                          probs = self.victim.get_prob([x_orig])
                      finally:
                          self.victim.clear_context()
                      y_orig = probs[0]
                  elif Tag("get_pred", "victim") in self.victim.TAGS:
                      self.victim.set_context(res["data"], None)
                      try:
                          preds = self.victim.get_pred([x_orig])
                      finally:
                          self.victim.clear_context()
                      y_orig = int(preds[0])
                  else:
                      raise RuntimeError("Invalid victim model")
              info = res["metrics"]
              info["Succeed"] = res["success"]
              if visualize:
                  if progress_bar:
                      visualizer(i + 1, x_orig, y_orig, x_adv, y_adv, info, tqdm_writer, self.tokenizer)
                  else:
                      visualizer(i + 1, x_orig, y_orig, x_adv, y_adv, info, sys.stdout.write, self.tokenizer)

              #list for tsv
              succeed_list.append(res["success"])
              num_queries_list.append(res["metrics"]["Victim Model Queries"])
              x_orig_list.append(x_orig)

              if res["success"]:
                x_adv_list.append(x_adv)
              else:
                x_adv_list.append("ATTACK_UNSUCCESSFUL")



          for kw, val in res["metrics"].items():
              if val is None:
                  continue

              if kw not in total_result_cnt:
                  total_result_cnt[kw] = 0
                  total_result[kw] = 0
              total_result_cnt[kw] += 1
              total_result[kw] += float(val)
      # End for

      summary = {}
      summary["Total Attacked Instances"] = total_inst
      summary["Successful Instances"] = success_inst
      summary["Attack Success Rate"] = success_inst / total_inst
      for kw in total_result_cnt.keys():
          if kw in ["Succeed"]:
              continue
          if kw in ["Query Exceeded"]:
              summary["Total " + kw] = total_result[kw]
          else:
              summary["Avg. " + kw] = total_result[kw] / total_result_cnt[kw]

      if visualize:
          result_visualizer(summary, sys.stdout.write)


      #saving tsv
      if tsv_file_path is not None:
        with open(tsv_file_path, 'w') as f:
          f.write('succeeded' + '\t' + 'num_queries' + '\t' + 'original_text' + '\t' + 'modified_text' + '\t'+ '\n') #header
          for success, num_queries, x_orig, x_adv in zip(succeed_list, num_queries_list, x_orig_list, x_adv_list):
            escaped_x_orig = x_orig.replace('\n', '\\n') #escaping newlines
            escaped_x_adv = x_adv.replace('\n', '\\n')
            f.write(str(success) + '\t' + str(num_queries) + '\t' + escaped_x_orig + '\t' + escaped_x_adv + '\t'+ '\n')

      return summary

## (optional) Mounting Google Drive


Steps to use mounted google drive:
1. create a folder in your local google drive (e.g. `incrediblAE_public_release`)  
2. download all directories from the download link (see [Download section above](https://colab.research.google.com/drive/1juHWIL44z8O3C5wDAE45vzlJgX51KI5D?authuser=3#scrollTo=eVVE2-64rKuS&line=3&uniqifier=1://)) and upload them to your google drive folder
3. create an empty subdirectory called `outputs` (`incredibleAE_public_release/outputs/`)

At this point, your google drive folder should have 6 subdirectories (C19, FC, HN, PR2, RD, and outputs)
4. uncomment code below, replacing path_to_mounted_dir with path to your folder (e.g. `/content/drive/My Drive/incrediblAE_public_release`)



In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# using_mounted_drive = True
# path_to_mounted_folder = '/content/drive/My Drive/incrediblAE_public_release'


You can also comment out the !gdown command in Downloading section, so the notebook doesn't redownload data each time you run it.

## Making custom attacker (token shuffler)

Here's an example of how to create a custom attack method.
Your attacker will need to subclass `OpenAttack.attackers.ClassificationAttacker`  

(See also OpenAttack framework docs: https://openattack.readthedocs.io/en/latest/)

In [7]:
import nltk
from nltk.corpus import stopwords, wordnet

nltk.download('stopwords')

nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\selin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\selin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\selin\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [29]:
'''
This example code shows how to design a customized attack model (that shuffles the tokens in the original sentence).
Taken from https://github.com/thunlp/OpenAttack/blob/master/examples/custom_attacker.py
'''

class MyAttacker(OpenAttack.attackers.ClassificationAttacker):
    @property
    def TAGS(self):
        # Returns tags to help OpenAttack check your parameters automatically
        return {self.lang_tag, Tag("get_pred", "victim")}

    def __init__(self, tokenizer=None, max_generations=10, population_size=20):
        if tokenizer is None:
            tokenizer = PunctTokenizer()
        self.tokenizer = tokenizer
        self.lang_tag = OpenAttack.utils.get_language([self.tokenizer])
        self.stop_words = set(stopwords.words('english'))
        self.max_generations = max_generations
        self.population_size = population_size
        self.homoglyphs = {
            'a': ['а', 'ɑ', 'а'],
            'e': ['е'],
            'o': ['о', 'ο', 'о'],
            'c': ['с', 'ϲ'],
            'p': ['р'],
            'x': ['х'],
            'y': ['у'],
            'i': ['і'],
            'l': ['ⅼ']
        }

    def attack(self, victim, input_, goal):
        try:
            original_tokens = self.tokenizer.tokenize(input_)
            population = [self.mutate(original_tokens) for _ in range(self.population_size)]

            for generation in range(self.max_generations):
                scored_population = [(individual, self.evaluate_fitness(individual, original_tokens, victim, goal)) for individual in population]
                population = [x[0] for x in sorted(scored_population, key=lambda x: x[1], reverse=True)]

                for individual in population:
                    x_new = self.tokenizer.detokenize(individual)
                    y_new = victim.get_pred([x_new])
                    if goal.check(x_new, y_new):
                        return x_new

                population = self.evolve_population(population, original_tokens, victim, goal)

            best_candidate = max(population, key=lambda ind: self.evaluate_fitness(ind, original_tokens, victim, goal))
            return self.tokenizer.detokenize(best_candidate)
        except Exception as e:
            print(e)
            return input_

    def evolve_population(self, population, original_tokens, victim, goal):
        new_population = []
        parent_candidates = sorted(population, key=lambda ind: self.evaluate_fitness(ind, original_tokens, victim, goal))[:len(population)//2]

        while len(new_population) < len(population):
            parent1, parent2 = random.sample(parent_candidates, 2)
            child1, child2 = self.crossover(parent1, parent2)
            new_population.append(self.mutate(child1))
            if len(new_population) < len(population):
                new_population.append(self.mutate(child2))

        return new_population
    
    def mutate(self, tokens, mutation_rate=0.1):
        mutated_tokens = []
        for token in tokens:
            if random.random() < mutation_rate:
                mutation_type = random.choice(["swap", "split"])  # Choose mutation type
                if mutation_type == "swap":
                    mutated_tokens.append(self.swap(token))
                elif mutation_type == "split":
                    mutated_tokens.append(self.split_word(token))
            else:
                mutated_tokens.append(token)
        return mutated_tokens


    def split_word(self, token):
        if isinstance(token, tuple):
            word = token[0]
        else:
            word = token

        if len(word) > 1:
            split_index = random.randint(1, len(word) - 1)
            first_part = word[:split_index]
            second_part = word[split_index:]
            if isinstance(token, tuple):
                return (first_part + ' ' + second_part, token[1])
            else:
                return first_part + ' ' + second_part
        else:
            return token


    def evaluate_fitness(self, individual, original_tokens, victim, goal):
        changes = sum(1 if orig != mod else 0 for orig, mod in zip(original_tokens, individual))
        extra_tokens = len(individual) - len(original_tokens) 

        adversarial_text = self.tokenizer.detokenize(individual)
        prediction = victim.get_pred([adversarial_text])

        if goal.check(adversarial_text, prediction):
            fitness_score = -(changes + extra_tokens)
        else:
            fitness_score = -1000 - changes - extra_tokens

        return fitness_score


    def crossover(self, parent1, parent2):
        possible_points = [i for i, token in enumerate(parent1) if ' ' not in token]
        if not possible_points:
            point = random.randint(1, len(parent1) - 1)
        else:
            point = random.choice(possible_points)

        child1 = parent1[:point] + parent2[point:]
        child2 = parent2[:point] + parent1[point:]
        return child1, child2
    
    def extra_bonus_for_splits(self, individual):
        return sum(1 for token in individual if ' ' in token)  # Example bonus for each split
    
    def select_parents(self, population, fitness_scores):
        weighted_population = [(individual, score + extra_bonus_for_splits(individual)) for individual, score in zip(population, fitness_scores)]
        sorted_population = sorted(weighted_population, key=lambda x: x[1], reverse=True)
        return [x[0] for x in sorted_population[:len(population) // 2]]  # Select the top half


    def is_better(self, individual1, individual2, original_tokens):
        changes1 = sum(1 for orig, mod in zip(original_tokens, individual1) if orig != mod)
        changes2 = sum(1 for orig, mod in zip(original_tokens, individual2) if orig != mod)

        if changes1 < changes2:
            return True
        return False

    def swap(self, token):
      if isinstance(token, tuple):
          word = token[0]
      else:
          word = token

      modified_word = ''.join([random.choice(self.homoglyphs[char]) if char in self.homoglyphs else char for char in word])

      if isinstance(token, tuple):
          return (modified_word, token[1])
      else:
          return modified_word


## Testing your attack

The code below will test MyAttacker (above) on the victim classifier, compute BODEGA score, and output results to /content/BODEGA/outputs.

WARNING: files in default output directory (/content/BODGEa/outputs) do not persist after you disconnect from the colab runtime session. To keep them, you can either:

1. download them manually or
2. set `out_dir` to a mounted Google Drive directory (will automatically save files to your google drive)



### Choose task + victim classifier

In [1]:
# determinism
random.seed(10)
torch.manual_seed(10)
np.random.seed(0)

# Change these variables to what you want
task = 'PR2' # PR2, HN, FC, RD, C19
victim_model = 'BERT' # BERT or BiLSTM
using_custom_attacker = True # change to False if you want to test out OpenAttack's pre-implemented attackers (e.g. BERTattack)
attack = 'genetic_3' # if using custom attack, this name can be whatever you want. If using pre-implemented attack, set to name of attacker ('BERTattack')

# misc variables - no need to change
targeted = False # this shared task evaluates performance in an untargeted scenario
visualize_adv_examples = True # prints adversarial samples as they are generated, showing the difference between original
using_first_n_samples = False # used when you want to evaluate on a subset of the full eval set.
first_n_samples = 20


NameError: name 'random' is not defined

### Run to evaluate attacker


In [30]:

if using_mounted_drive:
    data_path =  pathlib.Path(f"{path_to_mounted_folder}/{task}")
    model_path = pathlib.Path(f"{path_to_mounted_folder}/{task}/{victim_model}-512.pth")
    out_dir = pathlib.Path(f"{path_to_mounted_folder}/outputs")

else:
  data_path =  pathlib.Path(f"path-to-you-folder/incrediblAE_public_release/{task}")
  model_path = pathlib.Path(f"path-to-you-folder/incrediblAE_public_release/{task}/{victim_model}-512.pth")
  out_dir = pathlib.Path("path-to-you-folder/outputs")



RESULTS_FILE_NAME = 'results_' + task + '_' + str(targeted) + '_' + attack + '_' + victim_model + '.txt' #stores BODEGA metrics
SUBMISSION_FILE_NAME = 'submission_' + task + '_' + str(targeted) + '_' + attack + '_' + victim_model + '.tsv' #stores original and modified text, to be submitted to shared task organizers

results_path = out_dir / RESULTS_FILE_NAME if out_dir else None
submission_path = out_dir / SUBMISSION_FILE_NAME if out_dir else None

if out_dir:
    if (out_dir / RESULTS_FILE_NAME).exists():
      print(f"Existing results file found. This script will overwrite previous file: {str(results_path)}")
    if submission_path.exists():
      print(f"Existing submission file found. This script will overwrite previous file: {str(submission_path)}")




# Prepare task data
with_pairs = (task == 'FC' or task == 'C19')

# Choose device
print("Setting up the device...")

using_TF = (attack in ['TextFooler', 'BAE'])
if using_TF:
    # Disable GPU usage by TF to avoid memory conflicts
    import tensorflow as tf

    tf.config.set_visible_devices(devices=[], device_type='GPU')

if torch.cuda.is_available():
    print('using GPU')
    victim_device = torch.device("cuda")
    attacker_device = torch.device("cuda")
else:
    victim_device = torch.device("cpu")
    attacker_device = torch.device('cpu')

# Prepare victim
print("Loading up victim model...")
if victim_model == 'BERT':
    victim = VictimCache(model_path, VictimBERT(model_path, task, victim_device))
elif victim_model == 'BiLSTM':
    victim = VictimCache(model_path, VictimBiLSTM(model_path, task, victim_device))

# Load data
print("Loading data...")
test_dataset = Dataset.from_generator(readfromfile_generator,
                                      gen_kwargs={'subset': 'attack', 'dir': data_path, 'trim_text': True,
                                                  'with_pairs': with_pairs})
if not with_pairs:
    dataset = test_dataset.map(dataset_mapping)
    dataset = dataset.remove_columns(["text"])
else:
    dataset = test_dataset.map(dataset_mapping_pairs)
    dataset = dataset.remove_columns(["text1", "text2"])

dataset = dataset.remove_columns(["fake"])

# Filter data
if using_first_n_samples:
  dataset = dataset.select(range(first_n_samples))

if targeted:
    dataset = [inst for inst in dataset if inst["y"] == 1 and victim.get_pred([inst["x"]])[0] == inst["y"]]

print("Subset size: " + str(len(dataset)))

# Prepare attack
print("Setting up the attacker...")

# Necessary to bypass the outdated SSL certifiacte on the OpenAttack servers
with no_ssl_verify():
  if using_custom_attacker:
    attacker = MyAttacker()
  else:
    filter_words = OpenAttack.attack_assist.filter_words.get_default_filter_words('english') + [SEPARATOR_CHAR]
    if attack == 'PWWS':
        attacker = OpenAttack.attackers.PWWSAttacker(token_unk=UNK_TEXT, lang='english', filter_words=filter_words)
    elif attack == 'SCPN':
        os.environ["TOKENIZERS_PARALLELISM"] = "false"
        attacker = OpenAttack.attackers.SCPNAttacker(device=attacker_device)
    elif attack == 'TextFooler':
        attacker = OpenAttack.attackers.TextFoolerAttacker(token_unk=UNK_TEXT, lang='english',
                                                           filter_words=filter_words)
    elif attack == 'DeepWordBug':
        attacker = OpenAttack.attackers.DeepWordBugAttacker(token_unk=UNK_TEXT)
    elif attack == 'VIPER':
        attacker = OpenAttack.attackers.VIPERAttacker()
    elif attack == 'GAN':
        attacker = OpenAttack.attackers.GANAttacker()
    elif attack == 'Genetic':
        attacker = OpenAttack.attackers.GeneticAttacker(lang='english', filter_words=filter_words)
    elif attack == 'PSO':
        attacker = OpenAttack.attackers.PSOAttacker(lang='english', filter_words=filter_words)
    elif attack == 'BERTattack':
        attacker = OpenAttack.attackers.BERTAttacker(filter_words=filter_words, use_bpe=False, device=attacker_device)
    elif attack == 'BAE':
        attacker = OpenAttack.attackers.BAEAttacker(device=attacker_device, filter_words=filter_words)
    else:
        attacker = None

# Run the attack
print("Evaluating the attack...")
RAW_FILE_NAME = 'raw_' + task + '_' + str(targeted) + '_' + attack + '_' + victim_model + '.tsv'
raw_path = out_dir / RAW_FILE_NAME if out_dir else None

scorer = BODEGAScore(victim_device, task, align_sentences=True, semantic_scorer="BLEURT", raw_path = raw_path)
with no_ssl_verify():
    attack_eval = BodegaAttackEval(attacker, victim, language='english', metrics=[
        scorer  # , OpenAttack.metric.EditDistance()
    ])
    start = time.time()
    summary = attack_eval.eval_and_save_tsv(dataset, visualize=visualize_adv_examples, progress_bar=False, tsv_file_path = submission_path)
    end = time.time()
attack_time = end - start
attacker = None

# Remove unused stuff
victim.finalise()
del victim
gc.collect()
torch.cuda.empty_cache()
if "TOKENIZERS_PARALLELISM" in os.environ:
    del os.environ["TOKENIZERS_PARALLELISM"]

# Evaluate
start = time.time()
score_success, score_semantic, score_character, score_BODEGA= scorer.compute()
end = time.time()
evaluate_time = end - start

# Print results
print("Subset size: " + str(len(dataset)))
print("Success score: " + str(score_success))
print("Semantic score: " + str(score_semantic))
print("Character score: " + str(score_character))
print("BODEGA score: " + str(score_BODEGA))
print("Queries per example: " + str(summary['Avg. Victim Model Queries']))
print("Total attack time: " + str(attack_time))
print("Time per example: " + str((attack_time) / len(dataset)))
print("Total evaluation time: " + str(evaluate_time))

if out_dir:
  with open(results_path, 'w') as f:
      f.write("Subset size: " + str(len(dataset)) + '\n')
      f.write("Success score: " + str(score_success) + '\n')
      f.write("Semantic score: " + str(score_semantic) + '\n')
      f.write("Character score: " + str(score_character) + '\n')
      f.write("BODEGA score: " + str(score_BODEGA) + '\n')
      f.write("Queries per example: " + str(summary['Avg. Victim Model Queries']) + '\n')
      f.write("Total attack time: " + str(end - start) + '\n')
      f.write("Time per example: " + str((end - start) / len(dataset)) + '\n')
      f.write("Total evaluation time: " + str(evaluate_time) + '\n')

  print('-')
  print('Bodega metrics saved to', results_path)
  print('Submission file saved to', submission_path)

Setting up the device...
using GPU
Loading up victim model...
Victim caching: file found, loading...
Loading data...
Subset size: 416
Setting up the attacker...
Evaluating the attack...


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BleurtSPTokenizer'. 
The class this function is called from is 'BertTokenizer'.


Sample: 1 =====================================================================
Label: 0 (98.07%) --> 1 (90.69%)            |                                   
                                            |                                   
“ The U . S . Constitution clearly          |                                   
“ The U . S . Constitution clearly sta      |                                   
                                            |                                   
states that all citizens are born with      |                                   
tes    that all citizens are born with      | Running Time:            0.0050881
                                            | Query Exceeded:          no       
inalienable rights to be free from          | Victim Model Queries:    23       
inalienable rights to be free from          | BODEGA Score:            (later)  
                                            | Succeed:                 yes      
persecution by the government

Exception when evaluate data {'x': 'Ebola virus disease, formerly known as Ebola haemorrhagic fever, is a severe, often fatal illness in humans.', 'y': 1}
Traceback (most recent call last):
  File "c:\Users\selin\miniconda3\Lib\site-packages\OpenAttack\attack_eval\utils.py", line 10, in attack_process
    adversarial_sample = attacker(victim, data)
                         ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\selin\miniconda3\Lib\site-packages\OpenAttack\attackers\classification.py", line 33, in __call__
    raise RuntimeError("Check attacker result failed: result ([%d] %s) expect (%s%d)" % ( y_adv, adversarial_sample, "" if goal.targeted else "not ", goal.target))
RuntimeError: Check attacker result failed: result ([0] E bοlɑ v ir  us dіsеа sе , f οrmеrlу knоwn а s Ebоlа ha e morrhagic fеvе r , іs а sеvеrе , oft e n fata l il  lness і n humans .) expect (not 0)


Sample: 3 =====================================================================
Label: 0 (99.95%) --> Failed!               |                                   
                                            | Running Time:            0.036384 
Ebola virus disease , formerly known as     | Query Exceeded:          no       
Ebola haemorrhagic fever , is a severe ,    | Victim Model Queries:    622      
often fatal illness in humans .             | BODEGA Score:            (later)  
                                            | Succeed:                 no       
                                            |                                   
Sample: 4 =====================================================================
Label: 0 (99.97%) --> 1 (87.06%)            |                                   
                                            |                                   
So flagrant   was Ricca ’ s                 |                                   
So flagrant w as  Ríсϲа ’ s bе

...

Sample: 408 ===================================================================
Label: 0 (99.85%) --> Failed!               |                                   
                                            | Running Time:            0.033917 
Meanwhile , a third caravan of migrants ,   | Query Exceeded:          no       
this time from El Salvador , has reportedly | Victim Model Queries:    622      
already crossed into Mexico .               | BODEGA Score:            (later)  
                                            | Succeed:                 no       
                                            |                                   
Sample: 409 ===================================================================
Label: 0 (99.98%) --> 1 (84.70%)            |                                   
                                            |                                   
The Italian Catholic daily La Nuova Bussola |                                   
The Italian Catholic daily La 

Exception when evaluate data {'x': 'It’s hardest when it’s strongest.', 'y': 0}
Traceback (most recent call last):
  File "c:\Users\selin\miniconda3\Lib\site-packages\OpenAttack\attack_eval\utils.py", line 10, in attack_process
    adversarial_sample = attacker(victim, data)
                         ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\selin\miniconda3\Lib\site-packages\OpenAttack\attackers\classification.py", line 33, in __call__
    raise RuntimeError("Check attacker result failed: result ([%d] %s) expect (%s%d)" % ( y_adv, adversarial_sample, "" if goal.targeted else "not ", goal.target))
RuntimeError: Check attacker result failed: result ([0] I t ’ s hаrdеst whеn i t ’ s strоn gеst .) expect (not 0)


Sample: 412 ===================================================================
Label: 0 (99.98%) --> Failed!               |                                   
                                            | Running Time:            0.025341 
                                            | Query Exceeded:          no       
It ’ s hardest when it ’ s strongest .      | Victim Model Queries:    622      
                                            | BODEGA Score:            (later)  
                                            | Succeed:                 no       
                                            |                                   
Sample: 413 ===================================================================
Label: 0 (97.86%) --> 1 (68.03%)            |                                   
                                            |                                   
      Earlier this month , the U . S .      |                                   
Earli er      this month , the

Exception when evaluate data {'x': 'Email *', 'y': 0}
Traceback (most recent call last):
  File "c:\Users\selin\miniconda3\Lib\site-packages\OpenAttack\attack_eval\utils.py", line 10, in attack_process
    adversarial_sample = attacker(victim, data)
                         ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\selin\miniconda3\Lib\site-packages\OpenAttack\attackers\classification.py", line 33, in __call__
    raise RuntimeError("Check attacker result failed: result ([%d] %s) expect (%s%d)" % ( y_adv, adversarial_sample, "" if goal.targeted else "not ", goal.target))
RuntimeError: Check attacker result failed: result ([0] Emai l *) expect (not 0)


Sample: 415 ===================================================================
Label: 0 (99.99%) --> 1 (99.67%)            |                                   
                                            |                                   
The CAO and [ sergeant - at -    arms ]     |                                   
The CAO and [ sergeant - at - ar ms   ]     |                                   
                                            | Running Time:            0.0065072
worked with the Chairman to account for     | Query Exceeded:          no       
worked with thе Chаіrmɑn to account for h   | Victim Model Queries:    83       
                                            | BODEGA Score:            (later)  
   his   inventory , including the one      | Succeed:                 yes      
is inven tory      , including the оnе      |                                   
                                            |                                   
server .                     

Your output should look like this.
The custom attack has a very low BODEGA score, suggesting that the attack was not very successful (low success rate and low preservation of meaning).

VictimBERT on PR2:
```
Subset size: 416
Success score: 0.1778846153846154
Semantic score: 0.40792732766351186
Character score: 0.3001644500157
BODEGA score: 0.02308437726605881
Queries per example: 2.1778846153846154
Total attack time: 19.421820878982544
Time per example: 0.04668706942063112
Total evaluation time: 10.617336988449097
```

## Submission Files

Whenever you run an attack on a dataset, a submission_task.tsv file will be saved to your outputs directory. At the end of the test phase, you will need to submit your final attack's submission files to the shared task organisers for evaluation (1 for each dataset * num_victim_classifiers).

The submission file contains 4 pieces of information per attacked text:
1. was the attack successful
2. number of queries to victim model used to generate the adversarial sample
3. the original text
4. the adversarial text (or ATTACK_UNSUCCESSFUL if unsuccessful)

## Final tips:

### Using a subset of eval dataset
Testing your attack on the entire eval dataset can take a while. To speed things up, you can test on the first n samples of the dataset, by setting `using_first_n_samples` to `True`.  

### Running pre-implemented attacks

BODEGA supports a number of pre-existing attacks. Trying these might be useful if you want to:
- compare your performance with existing methods (also reported in the [BODEGA preprint](https://arxiv.org/abs/2303.08032))
- get inspiration from observing their substitutions

To use an existing attack requires only two changes to the code above:
1. set `using_custom_attacker` to `False`
2. set `attack` to the name of a supported attack
(`PWWS`, `SCPN`, `TextFooler`, `DeepWordBug`, `GAN`, `Genetic`, `PSO`, `BERTattack` or`BAE`)

Note that using `BAE` or `TextFooler` will require you to install additional dependencies since they rely on tensorflow:

- tensorflow >= 2.0.0
- tensorflow_hub

https://openattack.readthedocs.io/en/latest/quickstart/installation.html
